# Version 1

In [ ]:
### IMPORT PACKAGES #######################

import numpy as np
import scipy as sp
import sympy as spy
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.transforms
import pylab as plot
import networkx as nx
import random as rnd
import itertools as it
import math

from random import choice
from operator import add
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D


### SET rcParams #######################

mpl.rcParams['mathtext.fontset'] = 'stix'                       
mpl.rcParams['font.family'] = 'STIXGeneral'

params = {'legend.fontsize': 200}
mpl.rcParams.update(params)


### INIT PLANT POPS #######################

Start_Pop = 10 ** 6

N_A_Plants = [Start_Pop]
N_B1_Plants = [0]
N_B2_Plants = [0]
N_B3_Plants = [0]
N_C0_Plants = [0]
N_C1_Plants = [0]
N_C2_Plants = [0]
N_C3_Plants = [0]
N_D1_Plants = [0]
N_D2_Plants = [0]
N_E_Plants = [0]


### INIT NODES #######################

G = nx.MultiDiGraph()
for n in range(11):
    G.add_node(n, Plant_Pop = 0)

rnd.seed(2)

mapping={0:'$A$',1:'$B_1$',2:'$B_2$',3:'$B_3$',4:'$C_0$',\
         5:'$C_1$',6:'$C_2$',7:'$C_3$',8:'$D_1$',9:'$D_2$',10:'$E$'}

G = nx.relabel_nodes(G,mapping)

G.nodes['$A$']['Plant_Pop']= Start_Pop   ### STARTING PLANT POPULATION


### Probabilites / Percents #######################

# Forward Propagations
P_A_to_B1 = 0.25
P_A_to_B2 = 0.25
P_A_to_B3 = 0.25

P_B1_to_C0 = 0.25
P_B2_to_C0 = 0.25

P_B3_to_C1 = 0.25
P_C0_to_C1 = 0.25
P_C1_to_C2 = 0.25
P_C2_to_C3 = 0.25
P_C3_to_D1 = 0.99
P_D1_to_D2 = 0.99
P_D2_to_E = 0.99

# Interventions
P_Prev = 0.1
P_Mit = 0.1
P_Erad = 0.5
P_Cont = 0.95

# Percent of Motion
Per_Mo = 0.01

# Prob Death
P_A_D = 0
P_B1_D = 0
P_B2_D = 0
P_B3_D = 0
P_C0_D = 0
P_C1_D = 0
P_C2_D = 0
P_C3_D = 0
P_D1_D = 0
P_D2_D = 0
P_E_D = 0.0

# Percent Death
Per_De = 0.20


### INIT EDGES #######################

# Forward Propogation
G.add_edge('$A$', '$B_1$')
G.add_edge('$A$', '$B_2$')
G.add_edge('$A$', '$B_3$')
G.add_edge('$B_1$', '$C_0$')
G.add_edge('$B_2$', '$C_0$')
G.add_edge('$B_3$', '$C_1$')
G.add_edge('$C_0$', '$C_1$')
G.add_edge('$C_1$', '$C_2$')
G.add_edge('$C_2$', '$C_3$')
G.add_edge('$C_3$', '$D_1$')
G.add_edge('$D_1$', '$D_2$')
G.add_edge('$D_2$', '$E$')

FP = [('$A$', '$B_1$'),('$A$', '$B_2$'),('$A$', '$B_3$'),\
      ('$B_1$', '$C_0$'),('$B_2$', '$C_0$'),('$B_3$', '$C_1$'),('$C_0$', '$C_1$'),\
      ('$C_1$', '$C_2$'),('$C_2$', '$C_3$'),('$C_3$', '$D_1$'),('$D_1$', '$D_2$'),\
      ('$D_2$', '$E$')]
FP_colors = ['k'] * 12

# Prevention
G.add_edge('$B_1$', '$A$')
G.add_edge('$B_2$', '$A$')
Pr1 = [('$B_1$', '$A$')]
Pr2 = [('$B_2$', '$A$')]
Pr_colors = ['#45e8d5'] * 1

# Mitigation
G.add_edge('$E$', '$D_1$')
G.add_edge('$E$', '$D_2$')
Mt = [('$E$', '$D_1$'),('$E$', '$D_2$')]
Mt_colors = ['#5473ff'] * 2

# Eradication 1
G.add_edge('$E$', '$B_1$')
Erad1 = [('$E$', '$B_1$')]
Erad1_colors = ['#e0b14c'] * 1

# Eradication 2
G.add_edge('$E$', '$B_2$')
Erad2 = [('$E$', '$B_2$')]
Erad2_colors = ['#e0b14c'] * 1

# Containment
G.add_edge('$D_1$', '$C_1$')
G.add_edge('$D_1$', '$C_2$')
G.add_edge('$D_1$', '$C_3$')
G.add_edge('$D_2$', '$C_1$')
G.add_edge('$D_2$', '$C_2$')
G.add_edge('$D_2$', '$C_3$')
Cont = [('$D_1$', '$C_1$'),('$D_1$', '$C_2$'),('$D_1$', '$C_3$'),\
        ('$D_2$', '$C_1$'),('$D_2$', '$C_2$'),('$D_2$', '$C_3$')]
Cont_colors = ['#9469ea'] * 6


### BEGIN LOOPS #######################

T_Steps = 100

for t in range (0, T_Steps):
    
    c_edge = rnd.sample(list(G.edges()), 1)
    
    if c_edge[0] in FP:
        if c_edge[0] == ('$A$', '$B_1$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_A_to_B1:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$A$', '$B_2$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_A_to_B2:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$A$', '$B_3$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_A_to_B3:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$B_1$', '$C_0$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_B1_to_C0:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$B_2$', '$C_0$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_B2_to_C0:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$B_3$', '$C_1$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_B3_to_C1:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$C_0$', '$C_1$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_C0_to_C1:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$C_1$', '$C_2$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_C1_to_C2:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$C_2$', '$C_3$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_C2_to_C3:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$C_3$', '$D_1$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_C3_to_D1:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$D_1$', '$D_2$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_D1_to_D2:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        if c_edge[0] == ('$D_2$', '$E$'):
            if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
                if rnd.random() <= P_D2_to_E:
                    G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                    G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
        
    if c_edge[0] in Pr1 or Pr2:
        if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
            if rnd.random() <= P_Prev:
                G.nodes['$A$']['Plant_Pop'] += (G.nodes['$B_1$']['Plant_Pop']*Per_Mo)
                G.nodes['$A$']['Plant_Pop'] += (G.nodes['$B_2$']['Plant_Pop']*Per_Mo)
                G.nodes['$B_1$']['Plant_Pop'] -= (G.nodes['$B_1$']['Plant_Pop']*Per_Mo)
                G.nodes['$B_2$']['Plant_Pop'] -= (G.nodes['$B_2$']['Plant_Pop']*Per_Mo)
    
    if c_edge[0] in Mt:
        if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
            if rnd.random() <= P_Mit:
                G.nodes['$D_1$']['Plant_Pop'] += (G.nodes['$E$']['Plant_Pop']*Per_Mo)
                G.nodes['$D_2$']['Plant_Pop'] += (G.nodes['$E$']['Plant_Pop']*Per_Mo)
                G.nodes['$E$']['Plant_Pop'] -= (G.nodes['$B_1$']['Plant_Pop']*(2*Per_Mo))
    
    if c_edge[0] in Erad1 or Erad2:
        if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
            if rnd.random() <= P_Erad:
                G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                G.nodes[c_edge[0][1]]['Plant_Pop'] += (G.nodes[c_edge[0][0]]['Plant_Pop']*Per_Mo)
                G.nodes[c_edge[0][0]]['Plant_Pop'] -= (G.nodes[c_edge[0][0]]['Plant_Pop']*(2*Per_Mo))
    
    if c_edge[0] in Cont:
        if G.nodes[c_edge[0][0]]['Plant_Pop'] !=0:
            if rnd.random() <= P_Cont:
                G.nodes['$C_1$']['Plant_Pop'] += ((G.nodes['$D_1$']['Plant_Pop']*((1/3)*Per_Mo))+\
                                                  (G.nodes['$D_2$']['Plant_Pop']*((1/3)*Per_Mo)))
                G.nodes['$C_2$']['Plant_Pop'] += ((G.nodes['$D_1$']['Plant_Pop']*((1/3)*Per_Mo))+\
                                                  (G.nodes['$D_2$']['Plant_Pop']*((1/3)*Per_Mo)))
                G.nodes['$C_3$']['Plant_Pop'] += ((G.nodes['$D_1$']['Plant_Pop']*((1/3)*Per_Mo))+\
                                                  (G.nodes['$D_2$']['Plant_Pop']*((1/3)*Per_Mo)))
                G.nodes['$D_1$']['Plant_Pop'] -= (G.nodes['$D_1$']['Plant_Pop']*(Per_Mo))
                G.nodes['$D_2$']['Plant_Pop'] -= (G.nodes['$D_2$']['Plant_Pop']*(Per_Mo))
    
    if rnd.random() <= P_A_D:
        if G.nodes['$A$']['Plant_Pop'] > 0:
            G.nodes['$A$']['Plant_Pop'] -= (G.nodes['$A$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_B1_D:
        if G.nodes['$B_1$']['Plant_Pop'] > 0:
            G.nodes['$B_1$']['Plant_Pop'] -= (G.nodes['$B_1$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_B2_D:
        if G.nodes['$B_2$']['Plant_Pop'] > 0:
            G.nodes['$B_2$']['Plant_Pop'] -= (G.nodes['$B_2$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_B3_D:
        if G.nodes['$B_3$']['Plant_Pop'] > 0:
            G.nodes['$B_3$']['Plant_Pop'] -= (G.nodes['$B_3$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_C0_D:
        if G.nodes['$C_0$']['Plant_Pop'] > 0:
            G.nodes['$C_0$']['Plant_Pop'] -= (G.nodes['$C_0$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_C1_D:
        if G.nodes['$C_1$']['Plant_Pop'] > 0:
            G.nodes['$C_1$']['Plant_Pop'] -= (G.nodes['$C_1$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_C2_D:
        if G.nodes['$C_2$']['Plant_Pop'] > 0:
            G.nodes['$C_2$']['Plant_Pop'] -= (G.nodes['$C_2$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_C3_D:
        if G.nodes['$C_3$']['Plant_Pop'] > 0:
            G.nodes['$C_3$']['Plant_Pop'] -= (G.nodes['$C_3$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_D1_D:
        if G.nodes['$D_1$']['Plant_Pop'] > 0:
            G.nodes['$D_1$']['Plant_Pop'] -= (G.nodes['$D_1$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_D2_D:
        if G.nodes['$D_2$']['Plant_Pop'] > 0:
            G.nodes['$D_2$']['Plant_Pop'] -= (G.nodes['$D_2$']['Plant_Pop']*(Per_De))
    if rnd.random() <= P_E_D:
        if G.nodes['$E$']['Plant_Pop'] > 0:
            G.nodes['$E$']['Plant_Pop'] -= (G.nodes['$E$']['Plant_Pop']*(Per_De))
    
                
    ### READ OUT VALUES ####################

    N_A_Plants.append(G.nodes['$A$']['Plant_Pop'])
    N_B1_Plants.append(G.nodes['$B_1$']['Plant_Pop'])
    N_B2_Plants.append(G.nodes['$B_2$']['Plant_Pop'])
    N_B3_Plants.append(G.nodes['$B_3$']['Plant_Pop'])
    N_C0_Plants.append(G.nodes['$C_0$']['Plant_Pop'])
    N_C1_Plants.append(G.nodes['$C_1$']['Plant_Pop'])
    N_C2_Plants.append(G.nodes['$C_2$']['Plant_Pop'])
    N_C3_Plants.append(G.nodes['$C_3$']['Plant_Pop'])
    N_D1_Plants.append(G.nodes['$D_1$']['Plant_Pop'])
    N_D2_Plants.append(G.nodes['$D_2$']['Plant_Pop'])
    N_E_Plants.append(G.nodes['$E$']['Plant_Pop'])
    
    
    ### NODE COLOR CHANGES #######################
    
    Plant_Sum = 0

    for n in range(len(list(G.nodes()))):
        b = G.nodes[list(G.nodes())[n]]['Plant_Pop']
        Plant_Sum += b

    Nodes_List = [G.nodes[list(G.nodes())[n]]['Plant_Pop'] for n in range(11)]

    Node_Color_List = [x / Plant_Sum for x in Nodes_List]
    Node_Color_List1 = [x + 0.0 for x in Node_Color_List]
    

    ### EDGE THICKNESS CHANGES #######################


    N_As = [N_A_Plants, N_B1_Plants,N_B2_Plants,N_B3_Plants,\
            N_C0_Plants, N_C1_Plants, N_C2_Plants, N_C3_Plants,\
            N_D1_Plants, N_D2_Plants, N_E_Plants]

    def Edger(k,l):
        a = (\
         (N_As[k][len(N_As[k])-1]-N_As[k][len(N_As[k])-2]))

        b = (\
         (N_As[l][len(N_As[l])-1]-N_As[l][len(N_As[l])-2]))

        divider = Start_Pop
        multiplier = 4000 # around 3/400 for more movement
        c = (b-a)/divider
        d = abs(c)/2
        e = d * multiplier

        return(e)
    
    original = 17
    
    FP_lw = [original+Edger(0,1), original+Edger(0,2), original+Edger(0,3), original+Edger(1,4),\
             original+Edger(2,4), original+Edger(3,5), original+Edger(4,5), original+Edger(5,6),\
             original+Edger(6,7), original+Edger(7,8), original+Edger(8,9), original+Edger(9,10)]
    FP_As = [i * 6.5 for i in FP_lw]
    
    Pr1_lw = [original+Edger(1,0)]
    Pr1_As = [i * 6.5 for i in Pr1_lw]
    
    Pr2_lw = [original+Edger(2,0)]
    Pr2_As = [i * 6.5 for i in Pr2_lw]

    Mt_lw = [original+Edger(10,8), original+Edger(10,9)]
    Mt_As = [i * 6.5 for i in Mt_lw]

    Erad1_lw = [original+Edger(10,1)]
    Erad1_As = [i * 6.5 for i in Erad1_lw]

    Erad2_lw = [original+Edger(10,2)]
    Erad2_As = [i * 6.5 for i in Erad2_lw]
    
    Cont_lw = [original+Edger(8,5),original+Edger(8,6),original+Edger(8,7),\
              original+Edger(9,5),original+Edger(9,6),original+Edger(9,7)]
    Cont_As = [i * 6.5 for i in Cont_lw]
    

    ### DRAW GRAPH #######################
    
#    plt.ioff()
    pos = {'$A$':[-4,1.5],'$B_1$':[4,1.55],'$B_2$':[4,1.45],'$B_3$':[12,1.55]\
                ,'$C_0$':[12,1.45],'$C_1$':[20,1.5],'$C_2$':[27,1.5],'$C_3$':[34,1.5]\
                ,'$D_1$':[41,1.5],'$D_2$':[48,1.5],'$E$':[55.5,1.5]}

    plt.figure(figsize=(105, 58))

    # Nodes
    nx.draw_networkx_nodes(G,\
                          pos,\
                          node_size = 10.25**5,\
                          node_color = Node_Color_List1,
                          cmap = ('Greens'),
                          edgecolors=('#00441b'),\
                          linewidths = 15)
    
    # Node Labels
#     nx.draw_networkx_labels(G,\
#                            pos,\
#                            width_labels=False,\
#                            font_size = 100)

    # Forward Propagating Edges
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=FP_lw,\
                                   edgelist=FP,\
                                   edge_color=FP_colors,\
                                   connectionstyle='arc3, rad=0.0',\
                                   label='Forward Propagation')
    # Prevention 1
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Pr1_lw,\
                                   edgelist=Pr1,\
                                   edge_color=Pr_colors,\
                                   connectionstyle='arc3, rad=0.3',\
                                   label='Prevention')
    # Prevention 2                                                      
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Pr2_lw,\
                                   edgelist=Pr2,\
                                   edge_color=Pr_colors,\
                                   connectionstyle='arc3, rad=-0.3',\
                                   label='Prevention')
    # Mitigation
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Mt_lw,\
                                   edgelist=Mt,\
                                   edge_color=Mt_colors,\
                                   connectionstyle='arc3, rad=0.5',\
                                   label='Mitigation')
    # Eradication 1
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Erad1_lw,\
                                   edgelist=Erad1,\
                                   edge_color=Erad1_colors,\
                                   connectionstyle='arc3, rad=0.3')
    # Eradication 2
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Erad2_lw,\
                                   edgelist=Erad2,\
                                   edge_color=Erad2_colors,\
                                   connectionstyle='arc3, rad=-0.3',\
                                   label='Eradication')
    # Containment
    nx.nx_pylab.draw_networkx_edges(G, pos, arrowstyle='-|>',\
                                   arrowsize=140,\
                                   node_size = 10.5**5,\
                                   width=Cont_lw,\
                                   edgelist=Cont,\
                                   edge_color=Cont_colors,\
                                   connectionstyle='arc3, rad=-0.6',\
                                   label='Containment')


    ### LEGEND - DEPENDENT ON GRAPH DRAWING #######################

#     colors = ['k', '#45e8d5', '#5473ff', '#e0b14c','#9469ea']
#     lines = [Line2D([0], [0], color=c, linewidth=25, linestyle='-') for c in colors]
#     labels = ['Propagation', 'Prevention', 'Mitigation', 'Eradication', 'Containment']
#     plt.legend(lines, labels, loc=1, prop={'size': 120})
        
    colors = ['k', '#45e8d5', '#5473ff', '#e0b14c','#9469ea']
    lines = [Line2D([0], [0], color=c, linewidth=30, linestyle='-') for c in colors]
    labels = ['Propagation', 'Prevention', 'Mitigation', 'Eradication', 'Containment']
    plt.legend(lines, labels, prop={'size': 120},ncol=5, loc=(0.030,0.89), framealpha=1)
    


    ### LINES AND LABELS - DEPENDENT ON GRAPH DRAWING #######################

    qq = 1.38
    ii = 1.633

    aa = 0
    plt.plot([aa, aa], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    bb = 7.9
    plt.plot([bb, bb], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    cc = 16
    plt.plot([cc, cc], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    dd = 23.5
    plt.plot([dd, dd], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    ee = 30
    plt.plot([ee, ee], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    ff = 37.2
    plt.plot([ff, ff], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    gg = 44.2
    plt.plot([gg, gg], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')
    hh = 51.5
    plt.plot([hh, hh], [qq, ii], 'k-', lw=20, alpha=0.1, linestyle='dashed')

    plt.plot([51.5, 51.5], [1.351, 1.35], 'w', lw=20, alpha=1, linestyle='dashed')

    plt.text(-3.4,1.37,'Geography',fontsize=130,rotation=20,alpha=0.9)
    plt.text(4.5,1.364,' Captivity \nCultivation',fontsize=130,rotation=20,alpha=0.9)
    plt.text(13.3,1.374,'Survival',fontsize=130,rotation=20,alpha=0.9)
    plt.text(18.3,1.378,'Reproduction',fontsize=130,rotation=20,alpha=0.9)
    plt.text(25.2,1.365,' Sustainable \nReproduction',fontsize=130,rotation=20,alpha=0.9)
    plt.text(34.4,1.377,' Dispersal',fontsize=130,rotation=20,alpha=0.9)
    plt.text(40.2,1.365,'Sustainable \n  Dispersal',fontsize=130,rotation=20,alpha=0.9)
    plt.text(48.6,1.382,'Environmental',fontsize=130,rotation=20,alpha=0.9)

#     ### PRINT IMAGE #######################

# plt.savefig('/home/user/Desktop/Run.png', dpi=200) 
    


# Version 2

In [ ]:
%%time

### IMPORT PACKAGES #######################

import numpy as np
import scipy as sp
import sympy as spy
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.transforms
import pylab as plot
import random as rnd
import math
import networkx as nx
import random as rnd
import itertools as it

from random import choice
from operator import add
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D



### SET rcParams #######################

mpl.rcParams['mathtext.fontset'] = 'stix'                       
mpl.rcParams['font.family'] = 'STIXGeneral'

params = {'legend.fontsize': 200}
mpl.rcParams.update(params)



### INIT POP LISTS AND STARTING POPS #######################

N_I_All = []
N_E_All = []
N_S_All = []
N_N_All = []



I_pop = 60
E_pop = 100
S_pop = 10
N_pop = 100

rnd.seed(132)



### PROBS / VARIABLES #######################

adjustment = 1

r = 3.000 * adjustment
b = 4.000 * adjustment
p = 0.800 * adjustment
a = 0.800 * adjustment
gamma = 0.50 * adjustment
alpha = 0.24 * adjustment
beta = 0.500 * adjustment
c_1 = 0.30 * adjustment
c_2 = 0.40 * adjustment
c_3 = 0.10 * adjustment
k_I = 100 * adjustment
k_N = 200 * adjustment
mu_1 = 0.0 * adjustment
mu_2 = 0.0 * adjustment


def rates(I_pop, E_pop, S_pop, N_pop):
    r1 = abs(r * I_pop * (1-(I_pop/k_I)))
    r2 = abs((1-p)*beta*I_pop) 
    r3 = abs(p*beta*I_pop)
    r4 = abs(alpha * E_pop * (1 - (N_pop/(a + N_pop))))
    r5 = abs(mu_1 * S_pop)
    r6 = abs(c_1 * N_pop * S_pop)
    r7 = abs(gamma * S_pop)
    r8 = abs(b * N_pop * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop)/k_N)))
    r9 = abs(mu_2 * N_pop)
    return (r1, r2, r3, r4, r5, r6, r7, r8, r9)



### BEGIN LOOPS #######################

t_steps = 10**4
t_averaged = 1


for t_aved in range(t_averaged):


    N_I = []
    N_E = []
    N_S = []
    N_N = []

    for t in range(t_steps):

        for ii in range(int(I_pop)):
            if rnd.random() < r * (1 - (I_pop/k_I)):
                I_pop += 0.01
            if rnd.random() < (1 - p) * beta:
                I_pop -= 0.01
            if rnd.random() < p * beta:
                E_pop += 0.01
                I_pop -= 0.01         

        for ee in range(int(E_pop)):
            if rnd.random() < alpha * (1 - (N_pop / (a + N_pop))):
                S_pop += 0.01
                E_pop -= 0.01
        
        for ss in range(int(S_pop)):
            if S_pop >= 0:
                if rnd.random() < mu_1:
                    S_pop -= 0.01
            if S_pop >= 0:
                if rnd.random() < c_1 * N_pop:
                    S_pop -= 0.01
            if S_pop >= 0:
                if rnd.random() < gamma:
                    S_pop -= 0.01
                    E_pop += 0.01

        for nn in range(int(N_pop)):
            if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) > 0:
                if rnd.random() < b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)):
                    N_pop += 0.01
            if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) < 0:
                if rnd.random() < - b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)):
                    N_pop -= 0.01          
            if rnd.random() < mu_2:
                N_pop -= 0.01


        ### READ OUT VALUES ####################

        N_I.append(I_pop)
        N_E.append(E_pop)
        N_S.append(S_pop)
        N_N.append(N_pop)
        
    N_I_All.append(N_I)
    N_E_All.append(N_E)
    N_S_All.append(N_S)
    N_N_All.append(N_N)
    

    
### MERGE DATA ####################

arr1 = np.array(N_I_All)
N_I_All_Ave = list(np.mean(arr1, axis = 0))
arr2 = np.array(N_E_All)
N_E_All_Ave = list(np.mean(arr2, axis = 0))
arr3 = np.array(N_S_All)
N_S_All_Ave = list(np.mean(arr3, axis = 0))
arr4 = np.array(N_N_All)
N_N_All_Ave = list(np.mean(arr4, axis = 0))

arr = np.array(N_I_All)
N_I_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_E_All)
N_E_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_S_All)
N_S_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_N_All)
N_N_All_Std = list(np.std(arr, axis = 0))

N_I_All_Std_Plus = [a + b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Plus = [a + b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Plus = [a + b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Plus = [a + b for a, b in zip(N_N_All, N_N_All_Std)]

N_I_All_Std_Minus = [a - b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Minus = [a - b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Minus = [a - b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Minus = [a - b for a, b in zip(N_N_All, N_N_All_Std)]




    
### PLOT ####################

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_I_All_Ave, label='$I$  Population', linewidth=size1,alpha=alph)
#plt.plot(Steps, N_E_All_Ave, label='$E$  Population', linewidth=size1,alpha=alph)
#plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)
plt.plot(Steps, N_S_All_Ave, label='$S$  Population', color='green', linewidth=size1,alpha=alph)
plt.plot(Steps, N_N_All_Ave, label='$N$  Population', color='red', linewidth=size1,alpha=alph)

plt.xlabel('Time Steps', fontsize = '30')
plt.ylabel('Populations' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)

plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 19})





### PLOT ####################

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_E_All_Ave, label='$E$  Population', color='orange', linewidth=size1,alpha=alph)
# plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)


plt.xlabel('Time Steps', fontsize = '30')
plt.ylabel('Populations' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)

plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 19})



# Version 3

In [ ]:
%%time

### IMPORT PACKAGES #######################

import numpy as np
import scipy as sp
import sympy as spy
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.transforms
import pylab as plot
import random as rnd
import math
import networkx as nx
import random as rnd
import itertools as it

from random import choice
from operator import add
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D



### SET rcParams #######################

mpl.rcParams['mathtext.fontset'] = 'stix'                       
mpl.rcParams['font.family'] = 'STIXGeneral'

params = {'legend.fontsize': 200}
mpl.rcParams.update(params)



### INIT POPS #######################

N_I_All = []
N_E_All = []
N_S_All = []
N_N_All = []


I_pop = 60
E_pop = 100
S_pop = 10
N_pop = 100

rnd.seed(132)



### PROBS / VARIABLES #######################

adjustment = 1

r = 3.000 * adjustment
b = 4.000 * adjustment
p = 0.800 * adjustment
a = 0.800 * adjustment
gamma = 0.50 * adjustment
alpha = 0.24 * adjustment
beta = 0.500 * adjustment
c_1 = 0.30 * adjustment
c_2 = 0.40 * adjustment
c_3 = 0.10 * adjustment
k_I = 100 * adjustment
k_N = 200 * adjustment
mu_1 = 0.0 * adjustment
mu_2 = 0.0 * adjustment

base_prob = ( r * I_pop * (1-(I_pop/k_I)) ) + ( (1-p)*beta*I_pop )\
+ ( p*beta*I_pop ) + ( alpha * E_pop * (1-((N_pop)/(a + N_pop))) )\
+ ( mu_1 * S_pop ) + ( c_1 * N_pop * S_pop ) + ( gamma * S_pop )\
+ ( b * N_pop * (1 - ((N_pop + (c_2 * S_pop) + (c_3 * E_pop))/(k_N))) ) + ( mu_2 * N_pop )

fitness_change = 0

edge_prob_1 = (( r * I_pop * (1-(I_pop/k_I)) ) / base_prob) + (fitness_change*base_prob)
edge_prob_2 = (( (1-p)*beta*I_pop ) / base_prob) - (fitness_change*base_prob)
edge_prob_3 = (( p*beta*I_pop ) / base_prob) + (fitness_change*base_prob)
edge_prob_4 = (( alpha * E_pop * (1-((N_pop)/(a + N_pop))) ) / base_prob) + (fitness_change*base_prob)
edge_prob_5 = ((mu_1 * S_pop) / base_prob) - (fitness_change*base_prob)
edge_prob_6 = ((c_1 * N_pop * S_pop) / base_prob) - (fitness_change*base_prob)
edge_prob_7 = ((gamma * S_pop) / base_prob) - (fitness_change*base_prob)
edge_prob_8 = 0
edge_prob_9 = 0
edge_prob_10 = (( b * N_pop * (1 - ((N_pop + (c_2 * S_pop) + (c_3 * E_pop))/(k_N))) ) / base_prob)\
             + (fitness_change*base_prob)
edge_prob_11 = ((mu_2 * N_pop) / base_prob) # Constant


### BEGIN LOOPS #######################

t_steps = 10**5
t_averaged = 1


for t_aved in range(t_averaged):

    N_I = []
    N_E = []
    N_S = []
    N_N = []

    for t in range(t_steps):
        
        edge_choice = rnd.randint(1,11)
        
        if edge_choice == 1:
                I_pop += (edge_prob_1 * base_prob)
        
        if edge_choice == 2:
                I_pop -= (edge_prob_2 * base_prob)
                
        if edge_choice == 3:
                E_pop += (edge_prob_3 * base_prob)
                I_pop -= (edge_prob_3 * base_prob)
                
        if edge_choice == 4:
                S_pop += (edge_prob_4 * base_prob)
                E_pop -= (edge_prob_4 * base_prob)
                
        if edge_choice == 5:
                S_pop -= (edge_prob_5 * base_prob)
                
                
        if edge_choice == 6:
                S_pop -= (edge_prob_6 * base_prob)
                
                
        if edge_choice == 7:
                S_pop -= (edge_prob_7 * base_prob)
                E_pop += (edge_prob_7 * base_prob)
                
                
        if edge_choice == 10:
                N_pop += (edge_prob_10 * base_prob)
                
                
        if edge_choice == 11:
                N_pop -= (edge_prob_11 * base_prob)

  

        ### READ OUT VALUES ####################

        N_I.append(I_pop)
        N_E.append(E_pop)
        N_S.append(S_pop)
        N_N.append(N_pop)
        
    N_I_All.append(N_I)
    N_E_All.append(N_E)
    N_S_All.append(N_S)
    N_N_All.append(N_N)
    

    
    
    
### MERGE MULTI-RUN DATA ####################

arr1 = np.array(N_I_All)
N_I_All_Ave = list(np.mean(arr1, axis = 0))
arr2 = np.array(N_E_All)
N_E_All_Ave = list(np.mean(arr2, axis = 0))
arr3 = np.array(N_S_All)
N_S_All_Ave = list(np.mean(arr3, axis = 0))
arr4 = np.array(N_N_All)
N_N_All_Ave = list(np.mean(arr4, axis = 0))

arr = np.array(N_I_All)
N_I_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_E_All)
N_E_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_S_All)
N_S_All_Std = list(np.std(arr, axis = 0))
arr = np.array(N_N_All)
N_N_All_Std = list(np.std(arr, axis = 0))

N_I_All_Std_Plus = [a + b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Plus = [a + b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Plus = [a + b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Plus = [a + b for a, b in zip(N_N_All, N_N_All_Std)]

N_I_All_Std_Minus = [a - b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Minus = [a - b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Minus = [a - b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Minus = [a - b for a, b in zip(N_N_All, N_N_All_Std)]




    
### PLOT ####################

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_I_All_Ave, label='$I$  Population', linewidth=size1,alpha=alph)
#plt.plot(Steps, N_E_All_Ave, label='$E$  Population', linewidth=size1,alpha=alph)
#plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)
plt.plot(Steps, N_S_All_Ave, label='$S$  Population', color='green', linewidth=size1,alpha=alph)
plt.plot(Steps, N_N_All_Ave, label='$N$  Population', color='red', linewidth=size1,alpha=alph)

plt.xlabel('Time Steps', fontsize = '30')
plt.ylabel('Populations' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)

plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 19})





### PLOT #################### 

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_E_All_Ave, label='$E$  Population', color='orange', linewidth=size1,alpha=alph)
# plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)

plt.xlabel('Time Steps', fontsize = '30')
plt.ylabel('Populations' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)

plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 19})



# Version 4

In [ ]:
#### CYTHON #######################
# %load_ext line_profiler
%load_ext Cython



In [ ]:
%%time
%%cython -a

### IMPORT PACKAGES #######################

import time 

start = time.time() 

import numpy as np
import scipy as sp
import sympy as spy
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.transforms
import pylab as plot
import random as rnd
import math
import networkx as nx
import random as rnd
import itertools as it

from random import choice
from operator import add
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D




### SET rcParams #######################

mpl.rcParams['mathtext.fontset'] = 'stix'                       
mpl.rcParams['font.family'] = 'STIXGeneral'

params = {'legend.fontsize': 200}
mpl.rcParams.update(params)




### INIT POPS #######################

N_I_All = []
N_E_All = []
N_S_All = []
N_N_All = []

I_pop = 83.5
E_pop = 100
S_pop = 0
N_pop = 190

rnd.seed(133)




### PROBS AND VARIABLES #######################

fitness_adjust = 0.95

r_pre = 3.0
r = r_pre + (r_pre * fitness_adjust)

b_pre = 4.0
b = b_pre + (b_pre * 1)

p_pre = 0.800
p = p_pre + (p_pre * fitness_adjust)

a = 0.820 * fitness_adjust

gamma_pre = 0.50
gamma = gamma_pre - (gamma_pre * fitness_adjust)

alpha_pre = 0.24 
alpha = alpha_pre + (alpha_pre * fitness_adjust)

beta = 0.500 * fitness_adjust

c_1_pre = 0.30
c_1 = c_1_pre - (c_1_pre * fitness_adjust)

c_2_pre = 0.40 
c_2 = c_2_pre + (c_2_pre * fitness_adjust)

c_3_pre = 0.10
c_3 = c_3_pre + (c_3_pre * fitness_adjust)

k_I = 100 * fitness_adjust
k_N = 200 * fitness_adjust
mu_1 = 0.0 * fitness_adjust

mu_2_pre = 0.0
mu_2 = mu_2_pre + (mu_2_pre * 1)




### USEFUL FUNCTIONS #######################

move_def = 0.01

def do_rate(doing):
    
    global I_pop, E_pop, S_pop, N_pop
    
    if doing == 'I':
        if rnd.random() < r * (1 - (I_pop/k_I)):
            I_pop += move_def
        if rnd.random() < (1 - p) * beta:
            I_pop -= move_def
        if rnd.random() < p * beta:
            E_pop += move_def
            I_pop -= move_def   
            
    elif doing == 'E':
        if rnd.random() < alpha * (1 - (N_pop / (a + N_pop))):
            S_pop += move_def
            E_pop -= move_def
        
    elif doing == 'S':
        if S_pop >= 0:
            if rnd.random() < mu_1:
                S_pop -= move_def
        if S_pop >= 0:
            if rnd.random() < c_1 * N_pop:
                S_pop -= move_def
        if S_pop >= 0:
            if rnd.random() < gamma:
                S_pop -= move_def
                E_pop += move_def
        
    elif doing == 'N':
        if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) > 0:
            if rnd.random() < ( b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) ):
                N_pop += move_def
        if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) < 0:
            if rnd.random() < - ( b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) ):
                N_pop -= move_def          
        if rnd.random() < mu_2:
            N_pop -= move_def
            
            
def agent_compartment(I_pop, E_pop, S_pop, N_pop, agent):
    
    if agent < I_pop:
        return 'I'
    elif I_pop < agent < I_pop + E_pop:
        return 'E'
    elif I_pop + E_pop < agent < I_pop + E_pop + S_pop:
        return 'S'
    elif I_pop + E_pop + S_pop < agent < I_pop + E_pop + S_pop + N_pop:
        return 'N'
            

        

### LOOPS #######################


t_steps = round((500 / move_def))
t_averaged = 1


for t_aved in range(t_averaged):

    N_I = []
    N_E = []
    N_S = []
    N_N = []

    for t in range(t_steps):
        
        I_list = range(round(I_pop))
        E_list = range(round(I_pop), round(I_pop + E_pop))
        S_list = range(round(I_pop + E_pop), round(I_pop + E_pop + S_pop))
        N_list = range(round(I_pop + E_pop + S_pop), round(I_pop + E_pop + S_pop + N_pop))
        tot_list = list(range(round(I_pop + E_pop + S_pop + N_pop)))
        rnd.shuffle(tot_list)
        
        for agent in tot_list:
            function = agent_compartment(I_pop, E_pop, S_pop, N_pop, agent)
            do_rate(function)
  

        ### READ OUT VALUES ####################

        N_I.append(I_pop)
        N_E.append(E_pop)
        N_S.append(S_pop)
        N_N.append(N_pop)
        
    N_I_All.append(N_I)
    N_E_All.append(N_E)
    N_S_All.append(N_S)
    N_N_All.append(N_N)
    

    

### MERGE MULTI-RUN DATA ####################

arr1 = np.array(N_I_All)
N_I_All_Ave = list(np.mean(arr1, axis = 0))
arr2 = np.array(N_E_All)
N_E_All_Ave = list(np.mean(arr2, axis = 0))
arr3 = np.array(N_S_All)
N_S_All_Ave = list(np.mean(arr3, axis = 0))
arr4 = np.array(N_N_All)
N_N_All_Ave = list(np.mean(arr4, axis = 0))

arr = np.array(N_I_All)
N_I_All_Std = list(np.std(arr, axis = 0)/3)
arr = np.array(N_E_All)
N_E_All_Std = list(np.std(arr, axis = 0)/3)
arr = np.array(N_S_All)
N_S_All_Std = list(np.std(arr, axis = 0)/3)
arr = np.array(N_N_All)
N_N_All_Std = list(np.std(arr, axis = 0)/3)

N_I_All_Std_Plus = [a + b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Plus = [a + b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Plus = [a + b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Plus = [a + b for a, b in zip(N_N_All, N_N_All_Std)]

N_I_All_Std_Minus = [a - b for a, b in zip(N_I_All, N_I_All_Std)]
N_E_All_Std_Minus = [a - b for a, b in zip(N_E_All, N_E_All_Std)]
N_S_All_Std_Minus = [a - b for a, b in zip(N_S_All, N_S_All_Std)]
N_N_All_Std_Minus = [a - b for a, b in zip(N_N_All, N_N_All_Std)]


end = time.time()


print('Run Time (min) %s' %round(((end - start)/60),2))



In [ ]:
### PLOT I S N POPS OUT ####################

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_I_All_Ave, label='$I$  Population', linewidth=size1,alpha=alph)
#plt.plot(Steps, N_E_All_Ave, label='$E$  Population', linewidth=size1,alpha=alph)
#plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)
plt.plot(Steps, N_S_All_Ave, label='$S$  Population', color='green', linewidth=size1,alpha=alph)
plt.plot(Steps, N_N_All_Ave, label='$N$  Population', color='red', linewidth=size1,alpha=alph)

plt.xlabel('Time', fontsize = '30')
plt.ylabel('Population Densities' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)
plt.rc('font', size=20)

x1 = [0, 200 / move_def, 400 / move_def, 600 / move_def, 800 / move_def, 1000 / move_def]
squad = ['0','200','400','600','800','1000']

plt.xticks(x1, squad)



plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 23})




In [ ]:
### PLOT E POP OUT ####################

Steps = [i for i in range(len(N_E_All_Ave))]

plt.figure(figsize=(17, 9))

size1=5
alph=0.7
every_num = 9

plt.plot(Steps, N_E_All_Ave, label='$E$  Population', color='orange', linewidth=size1,alpha=alph)
# plt.fill_between(Steps, N_N_All_Std_Plus, N_S_All_Std_Minus, facecolor='#ffb668', alpha=0.5)


plt.xlabel('Time', fontsize = '30')
plt.ylabel('Population Densities' , fontsize = '25')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.tick_params(axis = 'both', direction = 'in', top = 1, right = 1, labelsize=25)
plt.rc('font', size=20)

x1 = [0, 200 / move_def, 400 / move_def, 600 / move_def, 800 / move_def, 1000 / move_def]
squad = ['0','200','400','600','800','1000']

plt.xticks(x1, squad)

plt.title('System Dynamics' , fontsize = '50')
#plt.ylim(-5**6,0.25*10**6)
#plt.xlim(-7,150)
plt.legend(loc='best' ,prop={'size': 23})





# Version 4: Multi-Runs

In [ ]:
#### FOR LINE PROFILING #######################
%load_ext line_profiler
%load_ext Cython


In [ ]:
%%time
%%cython -a



### IMPORT PACKAGES #######################

import time 

start = time.time() 

import numpy as np
import scipy as sp
import sympy as spy
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.transforms
import pylab as plot
import random as rnd
import math
import networkx as nx
import random as rnd
import itertools as it

from random import choice
from operator import add
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.lines import Line2D




### SET rcParams #######################

mpl.rcParams['mathtext.fontset'] = 'stix'                       
mpl.rcParams['font.family'] = 'STIXGeneral'

params = {'legend.fontsize': 200}
mpl.rcParams.update(params)




### INIT POP LISTS #######################

N_I_All_GRID = []
N_E_All_GRID = []
N_S_All_GRID = []
N_N_All_GRID = []

rnd.seed(133)


### USEFUL FUNCTIONS #######################

move_def = 0.01

def do_rate(doing):
    
    global I_pop, E_pop, S_pop, N_pop
    
    if doing == 'I':
        if rnd.random() < r * (1 - (I_pop/k_I)):
            I_pop += move_def
        if rnd.random() < (1 - p) * beta:
            I_pop -= move_def
        if rnd.random() < p * beta:
            E_pop += move_def
            I_pop -= move_def   
            
    elif doing == 'E':
        if rnd.random() < alpha * (1 - (N_pop / (a + N_pop))):
            S_pop += move_def
            E_pop -= move_def
        
    elif doing == 'S':
        if S_pop >= 0:
            if rnd.random() < mu_1:
                S_pop -= move_def
        if S_pop >= 0:
            if rnd.random() < c_1 * N_pop:
                S_pop -= move_def
        if S_pop >= 0:
            if rnd.random() < gamma:
                S_pop -= move_def
                E_pop += move_def
        
    elif doing == 'N':
        if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) > 0:
            if rnd.random() < ( b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) ):
                N_pop += move_def
        if b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) < 0:
            if rnd.random() < - ( b * (1 - ((N_pop + c_2 * S_pop + c_3 * E_pop) / k_N)) ):
                N_pop -= move_def          
        if rnd.random() < mu_2:
            N_pop -= move_def
            
            
def agent_compartment(I_pop, E_pop, S_pop, N_pop, agent):
    
    if agent < I_pop:
        return 'I'
    elif I_pop < agent < I_pop + E_pop:
        return 'E'
    elif I_pop + E_pop < agent < I_pop + E_pop + S_pop:
        return 'S'
    elif I_pop + E_pop + S_pop < agent < I_pop + E_pop + S_pop + N_pop:
        return 'N'
            


### TYPE EVERYTHING #######################

cdef long double fitness_adjust
cdef long double r_pre
cdef long double r
cdef long double p_pre
cdef long double p
cdef long double b_pre
cdef long double b
cdef long double a_pre
cdef long double a
cdef long double gamma_pre
cdef long double gamma
cdef long double alpha_pre
cdef long double alpha
cdef long double beta
cdef long double c_1_pre
cdef long double c_1
cdef long double c_2_pre
cdef long double c_2
cdef long double c_3_pre
cdef long double c_3
cdef long double k_I
cdef long double k_N
cdef long double mu_1
cdef long double mu_2_pre
cdef long double mu_2
cdef long int t
cdef long int t_steps
cdef long int t_aved
cdef long int t_averaged
cdef int fitnesses




### LOOPS #######################

t_steps = round((200 / move_def))
t_averaged = 1
fitnesses = 14


for fitness_adjust in np.linspace(0.001, 0.90, fitnesses):
    
    I_pop = 83.5
    E_pop = 100
    S_pop = 0
    N_pop = 190

    N_I_All = []
    N_E_All = []
    N_S_All = []
    N_N_All = []

    r_pre = 3.0
    r = r_pre + (r_pre * fitness_adjust)

    b_pre = 4.0
    b = b_pre + (b_pre * 1)

    p_pre = 0.800
    p = p_pre + (p_pre * fitness_adjust)

    a = 0.820 * fitness_adjust

    gamma_pre = 0.50
    gamma = gamma_pre - (gamma_pre * fitness_adjust)

    alpha_pre = 0.24 
    alpha = alpha_pre + (alpha_pre * fitness_adjust)

    beta = 0.500 * fitness_adjust

    c_1_pre = 0.30
    c_1 = c_1_pre - (c_1_pre * fitness_adjust)

    c_2_pre = 0.40 
    c_2 = c_2_pre + (c_2_pre * fitness_adjust)

    c_3_pre = 0.10
    c_3 = c_3_pre + (c_3_pre * fitness_adjust)

    k_I = 100 * fitness_adjust
    k_N = 200 * fitness_adjust
    mu_1 = 0.0 * fitness_adjust

    mu_2_pre = 0.0
    mu_2 = mu_2_pre + (mu_2_pre * 1)



    for t_aved in range(t_averaged):

        N_I = []
        N_E = []
        N_S = []
        N_N = []

        for t in range(t_steps):

            I_list = range(round(I_pop))
            E_list = range(round(I_pop), round(I_pop + E_pop))
            S_list = range(round(I_pop + E_pop), round(I_pop + E_pop + S_pop))
            N_list = range(round(I_pop + E_pop + S_pop), round(I_pop + E_pop + S_pop + N_pop))
            tot_list = list(range(round(I_pop + E_pop + S_pop + N_pop)))
            rnd.shuffle(tot_list)

            for agent in tot_list:
                function = agent_compartment(I_pop, E_pop, S_pop, N_pop, agent)
                do_rate(function)


            ### READ OUT VALUES ####################

            N_I.append(I_pop)
            N_E.append(E_pop)
            N_S.append(S_pop)
            N_N.append(N_pop)

        N_I_All.append(N_I)
        N_E_All.append(N_E)
        N_S_All.append(N_S)
        N_N_All.append(N_N)
        
    N_I_All_GRID.append(N_I_All)
    N_E_All_GRID.append(N_E_All)
    N_S_All_GRID.append(N_S_All)
    N_N_All_GRID.append(N_N_All)
        
    

### MERGE MULTI-RUN DATA ####################

N_I_All_Aved = []
N_E_All_Aved = []
N_S_All_Aved = []
N_N_All_Aved = []

cdef int N

for N in range(fitnesses):
    arr1 = np.array(N_I_All_GRID[N])
    I_Done = list(np.mean(arr1, axis = 0))
    N_I_All_Aved.append(I_Done)
    
    arr1 = np.array(N_E_All_GRID[N])
    E_Done = list(np.mean(arr1, axis = 0))
    N_E_All_Aved.append(E_Done)
    
    arr1 = np.array(N_S_All_GRID[N])
    S_Done = list(np.mean(arr1, axis = 0))
    N_S_All_Aved.append(S_Done)
    
    arr1 = np.array(N_N_All_GRID[N])
    N_Done = list(np.mean(arr1, axis = 0))
    N_N_All_Aved.append(N_Done)


end = time.time()


print('Run Time (min) %s' %round(((end - start)/60),2))




In [ ]:
### 30 STEP SETUP #######################

from mpl_toolkits.axes_grid1 import make_axes_locatable

plot_array_I = np.array(N_I_All_Aved)
plot_array_E = np.array(N_E_All_Aved)
plot_array_S = np.array(N_S_All_Aved)
plot_array_N = np.array(N_N_All_Aved)

fig, ax = plt.subplots(1,4, figsize=(40,10))

im0 = ax[0].imshow(plot_array_I, cmap='Greys', aspect = 1100, origin = 'lower') 
ax[0].set_title('Population $\mathbf{I}$ density given varying fitness' , fontsize = '30')
ax[0].set_ylabel('Fitness' , fontsize = '24')
ax[0].set_xlabel('Time (Time steps)', fontsize = '24')
ax[0].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$', '-' ,'-', '-', '-',  '-', '-','+$85\%$']
x_labels = ['','0','200', '400', '600', '800']
ax[0].set_yticks(np.arange(0, 31, 4.0))
ax[0].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[0].set_xticks(np.arange(0, 100000, 2))
ax[0].set_xticklabels(x_labels)
cbar0 = fig.colorbar(im0, ax=ax[0], pad=0.01, aspect=15, shrink=0.92)
cbar0.ax.tick_params(labelsize=20)

im1 = ax[1].imshow(plot_array_E, cmap='Greys', aspect = 1100, origin = 'lower') 
ax[1].set_title('Population $\mathbf{E}$ density given varying fitness' , fontsize = '30')
ax[1].set_ylabel('Fitness' , fontsize = '24')
ax[1].set_xlabel('Time (Time steps)', fontsize = '24')
ax[1].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$', '-' ,'-', '-', '-',  '-', '-','+$85\%$']
x_labels = ['','0','200', '400', '600', '800']
ax[1].set_yticks(np.arange(0, 31, 4.0))
ax[1].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[1].set_xticks(np.arange(0, 100000, 2))
ax[1].set_xticklabels(x_labels)
cbar1 = fig.colorbar(im1, ax=ax[1], pad=0.01, aspect=15, shrink=0.92)
cbar1.ax.tick_params(labelsize=20)

im2 = ax[2].imshow(plot_array_S, cmap='Greys', aspect = 1100, origin = 'lower') 
ax[2].set_title('Population $\mathbf{S}$ density given varying fitness' , fontsize = '30')
ax[2].set_ylabel('Fitness' , fontsize = '24')
ax[2].set_xlabel('Time (Time steps)', fontsize = '24')
ax[2].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$', '-' ,'-', '-', '-',  '-', '-','+$85\%$']
x_labels = ['','0','200', '400', '600', '800']
ax[2].set_yticks(np.arange(0, 31, 4.0))
ax[2].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[2].set_xticks(np.arange(0, 100000, 2))
ax[2].set_xticklabels(x_labels)
cbar2 = fig.colorbar(im2, ax=ax[2], pad=0.01, aspect=15, shrink=0.92)
cbar2.ax.tick_params(labelsize=20)

im3 = ax[3].imshow(plot_array_N, cmap='Greys', aspect = 1100, origin = 'lower') 
ax[3].set_title('Population $\mathbf{N}$ density given varying fitness' , fontsize = '30')
ax[3].set_ylabel('Fitness' , fontsize = '24')
ax[3].set_xlabel('Time (Time steps)', fontsize = '24')
ax[3].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$', '-' ,'-', '-', '-',  '-', '-','+$85\%$']
x_labels = ['','0','200', '400', '600', '800']
ax[3].set_yticks(np.arange(0, 31, 4.0))
ax[3].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[3].set_xticks(np.arange(0, 100000, 2))
ax[3].set_xticklabels(x_labels)
cbar3 = fig.colorbar(im3, ax=ax[3], pad=0.01, aspect=15, shrink=0.92)
cbar3.ax.tick_params(labelsize=20)


# fig.savefig('/home/user/Desktop/Peaks Go.png', dpi=500, bbox_inches = "tight") 

In [ ]:
### 14 STEP SETUP #######################

from mpl_toolkits.axes_grid1 import make_axes_locatable

# plot_array_I = np.array(N_I_All_Aved)
# plot_array_E = np.array(N_E_All_Aved)
# plot_array_S = np.array(N_S_All_Aved)
# plot_array_N = np.array(N_N_All_Aved)

fig, ax = plt.subplots(4,1, figsize=(20,30))
plt.subplots_adjust(hspace = 0.3)

im0 = ax[0].imshow(plot_array_I[:,0:30000], cmap='Greys', aspect = 700, origin = 'lower') 
ax[0].set_title('Population $\mathbf{I}$ density given varying fitness' , fontsize = '30')
ax[0].set_ylabel('Fitness' , fontsize = '24')
ax[0].set_xlabel('Time (Time steps)', fontsize = '24')
ax[0].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$' ,'-', '-', '-','-', '-', '+$90\%$']
x_labels = ['','0','50', '100', '150', '200','250']
ax[0].set_yticks(np.arange(0, 14, 2.0))
ax[0].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[0].set_xticks(np.arange(0, 100000, 2))
ax[0].set_xticklabels(x_labels)
cbar0 = fig.colorbar(im0, ax=ax[0], pad=0.01, aspect=15, shrink=0.92)
cbar0.ax.tick_params(labelsize=20)

im1 = ax[1].imshow(plot_array_E[:,0:30000], cmap='Greys', aspect = 700, origin = 'lower') 
ax[1].set_title('Population $\mathbf{E}$ density given varying fitness' , fontsize = '30')
ax[1].set_ylabel('Fitness' , fontsize = '24')
ax[1].set_xlabel('Time (Time steps)', fontsize = '24')
ax[1].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$' ,'-', '-', '-' ,'-', '-', '+$90\%$']
x_labels = ['','0','50', '100', '150', '200','250']
ax[1].set_yticks(np.arange(0, 14, 2.0))
ax[1].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[1].set_xticks(np.arange(0, 100000, 2))
ax[1].set_xticklabels(x_labels)
cbar1 = fig.colorbar(im1, ax=ax[1], pad=0.01, aspect=15, shrink=0.92)
cbar1.ax.tick_params(labelsize=20)

im2 = ax[2].imshow(plot_array_S[:,0:30000], cmap='Greys', aspect = 700, origin = 'lower') 
ax[2].set_title('Population $\mathbf{S}$ density given varying fitness' , fontsize = '30')
ax[2].set_ylabel('Fitness' , fontsize = '24')
ax[2].set_xlabel('Time (Time steps)', fontsize = '24')
ax[2].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$' ,'-', '-', '-' ,'-', '-', '+$90\%$']
x_labels = ['','0','50', '100', '150', '200','250']
ax[2].set_yticks(np.arange(0, 14, 2.0))
ax[2].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[2].set_xticks(np.arange(0, 100000, 2))
ax[2].set_xticklabels(x_labels)
cbar2 = fig.colorbar(im2, ax=ax[2], pad=0.01, aspect=15, shrink=0.92)
cbar2.ax.tick_params(labelsize=20)

im3 = ax[3].imshow(plot_array_N[:,0:30000], cmap='Greys', aspect = 700, origin = 'lower') 
ax[3].set_title('Population $\mathbf{N}$ density given varying fitness' , fontsize = '30')
ax[3].set_ylabel('Fitness' , fontsize = '24')
ax[3].set_xlabel('Time (Time steps)', fontsize = '24')
ax[3].tick_params(axis = 'both', direction = 'in', top = 11, right = 1, labelsize=21)
y_labels = ['$Unif$' ,'-', '-', '-' ,'-', '-', '+$90\%$']
x_labels = ['','0','50', '100', '150', '200','250']
ax[3].set_yticks(np.arange(0, 14, 2.0))
ax[3].set_yticklabels(y_labels, rotation=45, ha = 'right')
#ax[3].set_xticks(np.arange(0, 100000, 2))
ax[3].set_xticklabels(x_labels)
cbar3 = fig.colorbar(im3, ax=ax[3], pad=0.01, aspect=15, shrink=0.92)
cbar3.ax.tick_params(labelsize=20)


# fig.savefig('/home/user/Desktop/Peaks Go.png', dpi=500, bbox_inches = "tight") 




In [ ]:
### PICKLE OUTPUTS #######################

import os
import pickle

os.chdir('/home/user/Desktop/')

# plot_array_I = np.array(N_I_All_Aved)
# plot_array_E = np.array(N_E_All_Aved)
# plot_array_S = np.array(N_S_All_Aved)
# plot_array_N = np.array(N_N_All_Aved)


def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)        
def load_obj(name):
    with open(name, 'rb') as f:
        return pickle.load(f)
    
    
    
save_obj(plot_array_I, 'plot_array_I.pkl')
save_obj(plot_array_E, 'plot_array_E.pkl')
save_obj(plot_array_S, 'plot_array_S.pkl')
save_obj(plot_array_N, 'plot_array_N.pkl')


# plot_array_I = load_obj('plot_array_I.pkl')
# plot_array_E = load_obj('plot_array_E.pkl')
# plot_array_S = load_obj('plot_array_S.pkl')
# plot_array_N = load_obj('plot_array_N.pkl')


